This notebook shows primary color analysis of color image using K-Means algorithm.
The output are N primary colors and their corresponding percentage.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

import webcolors
from sklearn.metrics import mean_squared_error
import pandas as pd
import plotly.express as px
import colorgram

In [ ]:
def rgb2name(c):
    h_color = "#{:02x}{:02x}{:02x}".format(int(c[0]), int(c[1]), int(c[2]))
    try:
        nm = webcolors.hex_to_name(h_color, spec="css3")
    except ValueError:
        rms_lst = []
        for img_clr, img_hex in webcolors.CSS3_NAMES_TO_HEX.items():
            cur_clr = webcolors.hex_to_rgb(img_hex)
            rmse = np.sqrt(mean_squared_error(c, cur_clr))
            rms_lst.append(rmse)

        closest_color = rms_lst.index(min(rms_lst))

        nm = list(webcolors.CSS3_NAMES_TO_HEX.items())[closest_color][0]
    return nm

In [ ]:
n_cluster = 20


def analyze_image(image_path, n_extract=10):
    colors = colorgram.extract(image_path, n_extract)
    output_dict = {}
    # average color in each bin and save bin length
    for color in colors:
        label = f"{rgb2name(color.rgb)} ({color.rgb})"
        output_dict[label] = {
            "color_mean": color.rgb,
            "hex_mean": webcolors.rgb_to_hex(color.rgb),
            "label": label + f" {round(color.proportion,2)}",
            "color_name": rgb2name(color.rgb),
            "bin_length": color.proportion,
            "percentage": color.proportion,
        }

    df = pd.DataFrame(output_dict)
    df = df.reindex(sorted(df.columns), axis=1)
    return df

In [ ]:
def show_piechart(df, n_max=-1):
    if n_max == -1:
        n_max = len(df.T)

    df = df.T.sort_values(by="bin_length", ascending=False).head(n_max)

    color_map = {
        color: hex_mean for color, hex_mean in zip(df["label"], df["hex_mean"])
    }
    fig = px.pie(
        df,
        values="bin_length",
        names="label",
        title="Color analysis",
        color="label",
        color_discrete_map=color_map,
        hole=0.3,
    )
    fig.show()

In [ ]:
image_path = "../../documents-export-2023-04-27/images/102435S_2_eng.png"


image = cv2.imread(image_path)
# BGR-->RGB cv to matplotlib show
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
df = analyze_image(image_path, 10)
show_piechart(df)

In [ ]:
df